In [3]:
!pip install import_ipynb
!pip install tensorflow-datasets

     ---------------------------------------- 4.2/4.2 MB 20.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 48.8/48.8 KB ? eta 0:00:00
     ---------------------------------------- 86.9/86.9 KB ? eta 0:00:00
     ---------------------------------------- 241.5/241.5 KB ? eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21502 sha256=533756df2a8b846b4030c60f4cdb24991d72626ee9f6eb3a589b4ecb531316d6
  Stored in directory: c:\users\administrator\appdata\local\pip\cache\wheels\e1\e8\83\ddea66100678d139b14bc87692ece57c6a2a937956d2532608
Successfully built promise


In [21]:
# data를 옮기고
# from tensorflow_datasets.image_classification.resisc45 import Resisc45
!cd resisc45_NEW.py
!tfds build  # tfds 데이터셋 빌드

디렉터리 이름이 올바르지 않습니다.
2022-05-08 23:10:08.829510: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-08 23:10:08.829560: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-05-08 23:10:12.188222: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-05-08 23:10:12.188267: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-08 23:10:12.191764: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: DESKTOP-2LUS9NO
2022-05-08 23:10:12.191829: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: DESKTOP-2LUS9NO
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\runpy.py", lin

In [13]:
 # .py로 매번 수정하기 귀찮아서 ipynb 파일로 import가능하게 만드는 라이브러리
import import_ipynb 

import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import models
import resisc45_NEW

GoogLeNet = models.InceptionNet

In [14]:
#하이퍼 파라미터
batch_size = 128
num_epoch = 100
train_accuracies = []
test_accuracies = []


In [16]:
dataset, info = tfds.load('resisc45_NEW',as_supervised = True, with_info = True)
dataset_test, dataset_train = dataset['test'], dataset['train']

AssertionError: You must download the dataset manually from http://www.escience.cn/people/JunweiHan/NWPU-RESISC45.html, extract it, and place it in C:\Users\Administrator\tensorflow_datasets\downloads\manual.

In [11]:


# dataset, info = tfds.load('fashion_mnist', as_supervised = True, with_info = True)
# dataset_test, dataset_train = dataset['test'], dataset['train']


dataset_train = dataset_train.map(convert_types).shuffle(10000).batch(batch_size)
dataset_test = dataset_test.map(convert_types).batch(batch_size)

datagen = ImageDataGenerator(rotation_range=10, horizontal_flip=True, zoom_range=0.1)

# GoogleNet 객체 생성
model = GoogLeNet((28, 28, 1), 10)
model.build(input_shape = (None, 28, 28, 1))
model.summary()

# Prepare Taining 라벨링

############## #사용자 루프 모델 학습 ######
# model.compile() 기능을 사용자가 직접 수동으로 하는 것
#def compile_by_users():
    
# 입력값에 대해 원핫라벨링한 뒤, 교차 엔트로피에 통과하여 loss를 검사
loss_object = keras.losses.SparseCategoricalCrossentropy()
# 최적화는 adam
optimizer = keras.optimizers.Adam()

train_loss = keras.metrics.Mean(name = 'train_loss')
train_accuracy = keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')
test_loss = keras.metrics.Mean(name = 'test_loss')
test_accuracy = keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')
    


#compile_by_users()

def train_step(image, label): # 훈련 이미지, 훈련 레이블
    with tf.GradientTape() as tape:
        
        predictions = model(image)
        loss = loss_object(label, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(label, predictions)
    
def test_step(image, label): # 훈련 이미지, 훈련 레이블
        
    predictions = model(image)
    loss = loss_object(label, predictions)
    
    test_loss(loss)
    test_accuracy(label, predictions)
    


AssertionError: Manual directory C:\Users\Administrator\tensorflow_datasets\downloads\manual does not exist or is empty. Create it and download/extract dataset artifacts in there using instructions:
Dataset can be downloaded from OneDrive:
https://1drv.ms/u/s!AmgKYzARBl5ca3HNaHIlzp_IXjs
After downloading the rar file, please extract it to the manual_dir.

In [16]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [ ]:
for epoch in range(num_epoch):
    i = 0
    for image, label in dataset_train:
        for _image, _label in datagen.flow(image, label, batch_size = batch_size):
            printProgressBar(i + 1, len(dataset_train),  prefix = 'Progress:', suffix = 'Complete', length = 50)
            # Data Augmentabtion된 훈련 데이터를 _image, 훈련 레이블을 _label로 저장
            # Data Augmentabtion이란 데이터셋을 여러 방법으로 변경하여 데이터셋 개수를 늘리는 방법이다.
            # 회전을 시킨다거나, 임의의 범위를 잘라낸다건, 하나의 소르를 가지고 데이터를 우려내는 방법이다.
            train_step(_image, _label) # 모델에 data Augmentation된 훈련 데이터셋을 넣음
            i+=1
            break
    # 훈련 데이터 셋에 대한 학습 종료

    for test_image, test_label in dataset_test:
        test_step(test_image, test_label)
        # 시험 데이터 셋 테스트 종료
        
    train_accuracies.append(train_accuracy.result())
    test_accuracies.append(test_accuracy.result())
    
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1, train_loss.result(), train_accuracy.result() * 100,
                          test_loss.result(), test_accuracy.result() * 100))